In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns

In [92]:
output_dir = "../.."

In [32]:
all_scores = list(Path("../../outputs/encoders").glob("*/*/scores.csv"))
all_scores_keys = [
    (x.parent.name, x.parents[1].name)
    for x in all_scores
]

In [79]:
all_scores_keys

[('EC195', 'syllable'),
 ('EC196', 'syllable'),
 ('EC152', 'syllable'),
 ('EC152', 'random8'),
 ('EC196', 'random8'),
 ('EC195', 'random8'),
 ('EC196', 'next_phoneme'),
 ('EC195', 'next_phoneme'),
 ('EC152', 'next_phoneme'),
 ('EC152', 'phoneme8'),
 ('EC196', 'phoneme8'),
 ('EC195', 'phoneme8'),
 ('EC196', 'phoneme'),
 ('EC195', 'phoneme'),
 ('EC152', 'phoneme'),
 ('EC152', 'baseline'),
 ('EC196', 'baseline'),
 ('EC195', 'baseline')]

In [94]:
all_scores_df = pd.concat([
    pd.read_csv(x) for x in all_scores
], keys=all_scores_keys, names=["subject", "model"]).reset_index(level=2, drop=True)
all_scores_df

fold  output_dim     score output_name
subject model                                           
EC195   syllable     0           0 -0.002152         GP1
        syllable     1           0 -0.000462         GP1
        syllable     2           0 -0.000690         GP1
        syllable     0           1 -0.000266         GP2
        syllable     1           1 -0.001244         GP2
...                ...         ...       ...         ...
        baseline     1         254 -0.000045       GA127
        baseline     2         254  0.000737       GA127
        baseline     0         255 -0.002221       GA128
        baseline     1         255 -0.000587       GA128
        baseline     2         255  0.000243       GA128

[13824 rows x 4 columns]

In [23]:
all_electrodes = list(Path("../../outputs/encoders").glob("*/*/electrodes.csv"))
all_electrodes_df = pd.concat(
    [pd.read_csv(x) for x in all_electrodes],
    keys=[x.parent.name for x in all_electrodes],
    names=["subject"]
).set_index("label", append=True).reset_index(level=1, drop=True)
all_electrodes_df = all_electrodes_df[~all_electrodes_df.index.duplicated()]
all_electrodes_df

long_name   type                          roi
subject label                                                                
EC195   GP1                PosteriorGrid1   grid             inferiortemporal
        GP2                PosteriorGrid2   grid             inferiortemporal
        GP3                PosteriorGrid3   grid             inferiortemporal
        GP4                PosteriorGrid4   grid               middletemporal
        GP5                PosteriorGrid5   grid               middletemporal
...                                   ...    ...                          ...
EC152   HD4    HippocampusDepthElectrode4  depth  Right-Cerebral-White-Matter
        HD5    HippocampusDepthElectrode5  depth  Right-Cerebral-White-Matter
        HD6    HippocampusDepthElectrode6  depth  Right-Cerebral-White-Matter
        HD7    HippocampusDepthElectrode7  depth  Right-Cerebral-White-Matter
        HD8    HippocampusDepthElectrode8  depth                      Unknown

[902 rows x 3 columns]

In [95]:
all_scores_df.to_csv(Path(output_dir) / "all_encoding_scores.csv")
all_electrodes_df.to_csv(Path(output_dir) / "all_electrodes.csv")

## Comparison across subject

In [86]:
reg_data = all_improvements_df.loc[(slice(None), "syllable", "baseline")].reset_index()
reg_data["score"] -= reg_data.score.mean()
reg_data["score"] /= reg_data.score.std()
reg_data["subj_elec"] = reg_data.subject + "_" + reg_data.output_name

In [87]:
import statsmodels.formula.api as smf

In [88]:
model = smf.mixedlm("score ~ model", data=reg_data, groups=reg_data.subj_elec)

In [89]:
model_fit = model.fit()

In [90]:
model_fit.params

Intercept            -0.000072
model[T.syllable]     0.000145
Group Var            11.957198
dtype: float64

In [91]:
model_fit.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:              MixedLM  Dependent Variable:  score     
No. Observations:   4608     Method:              REML      
No. Groups:         768      Scale:               0.0773    
Min. group size:    6        Log-Likelihood:      -2290.2558
Max. group size:    6        Converged:           Yes       
Mean group size:    6.0                                     
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         -0.000    0.035 -0.002 0.998 -0.069  0.069
model[T.syllable]  0.000    0.008  0.018 0.986 -0.016  0.016
Group Var          0.924    0.188                           
============================================================

"""